## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - ],Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [5]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by `TYPE`

In [6]:
i = 3
print CATEGORY_GROUPS_IN_QUESTION[i]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[i])]
df_orig.shape

['Sidewalk Repair', 'Sidewalk Repair (Make Safe)']


(12512, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [7]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(12473, 40)

## Remove `TYPE` col

In [8]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [ ]:
['race_black',
 'race_asian',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_ssi',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'rent_std_dev',
 'income_std_dev',
 'Source_Constituent Call']

In [124]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'SubmittedPhoto', 'Property_Type', 'Source']
cols_census = [
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_food_stamps',
     'housing',
     'housing_std_dev',
     'value_std_dev',
     'rent',
]
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [125]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(12473, 18)

## Removing NAs for cols like `school_std_dev`

In [126]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

12    housing_std_dev
13      value_std_dev
Name: index, dtype: object


In [127]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(12043, 18)

## Dummify

In [128]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'housing']

In [129]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

Intersection is baseline 0 3
Self Service is baseline 1 3
own is baseline 2 3


In [86]:
df_dummified.shape

(12043, 19)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [16]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

poverty_pop_w_food_stamps          4.435174
poverty_pop_below_poverty_level    3.294339
earned_income_per_capita           2.900774
Source_Citizens Connect App        2.707832
Source_Constituent Call            2.564019
income                             2.461304
poverty_pop_w_ssi                  2.151635
race_black                         2.108312
school                             2.019925
race_hispanic                      1.966331
rent_std_dev                       1.837870
school_std_dev                     1.762409
income_std_dev                     1.672045
housing_std_dev                    1.661818
poverty_pop_w_public_assistance    1.613030
rent                               1.576112
bedroom                            1.538475
value_std_dev                      1.516026
race_asian                         1.451496
value                              1.448055
queue_wk                           1.216030
bedroom_std_dev                    1.184542
race_other                      

## Running model

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [18]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [21]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [28]:
params = {'lassocv__alphas': make_alphas(-2, 2)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

Fitting 1 folds for each of 9 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.7s finished


,0,1,2,3,4,5,6,7,8
mean_test_score,0.261992,0.25724,0.229553,0.146099,-0.000708715,-0.000708715,-0.000708715,-0.000708715,-0.000708715
mean_train_score,0.25045,0.245236,0.219893,0.14113,0,0,0,0,0
param_lassocv__alphas,[0.01],[0.03],[0.1],[0.3],[1.0],[3.0],[10.0],[30.0],[100.0]


In [29]:
model.best_params_

{'lassocv__alphas': [0.01]}

In [30]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'10 cols go to zero out of 27'

In [31]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['race_black',
 'race_asian',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_ssi',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'rent_std_dev',
 'income_std_dev',
 'Source_Constituent Call']

## Use subsetted cols to run lin reg

In [130]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [131]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [132]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [42]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.258
Model:,OLS,Adj. R-squared:,0.256
Method:,Least Squares,F-statistic:,166.9
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,0.00
Time:,08:11:56,Log-Likelihood:,-13641.
No. Observations:,9634,AIC:,2.732e+04
Df Residuals:,9613,BIC:,2.748e+04
Df Model:,20,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [133]:
y_pred = est.predict(X_test)

In [134]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.23872377354105162

In [135]:
mean_squared_error(y_test, y_pred)**0.5

1.008450882506903

## Interpreting model

Which features are most associated with completion time?

In [136]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results['coef_abs'] = pd.np.abs(df_results.coef)
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results.shape

(18, 7)

In [137]:
df_results

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,2.058200e+00,7.400000e-02,27.818,0.000,1.913 2.203,2.058200e+00
2,isdescription[T.True],1.203800e+00,4.500000e-02,27.040,0.000,1.117 1.291,1.203800e+00
14,queuewkopen,6.000000e-04,3.390000e-05,18.345,0.000,0.001 0.001,6.000000e-04
5,racehispanic,-4.096000e-01,7.900000e-02,-5.192,0.000,-0.564 -0.255,4.096000e-01
13,queuewk,-1.766000e-05,2.060000e-06,-8.586,0.000,-2.17e-05 -1.36e-05,1.766000e-05
16,SourceCitizensConnectApp,-2.137000e-01,3.600000e-02,-5.971,0.000,-0.284 -0.144,2.137000e-01
9,povertypopwfoodstamps,5.576000e-01,1.170000e-01,4.771,0.000,0.329 0.787,5.576000e-01
11,valuestddev,-1.719400e+00,5.040000e-01,-3.415,0.001,-2.706 -0.732,1.719400e+00
6,raceother,-7.290000e-01,2.380000e-01,-3.061,0.002,-1.196 -0.262,7.290000e-01
12,rent,4.833000e-05,1.780000e-05,2.710,0.007,1.34e-05 8.33e-05,4.833000e-05


In [110]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('raceasian', 49),
 ('earnedincomepercapita', 48.897),
 ('housingstddev', 48.871),
 ('SubmittedPhoto', 48.838),
 ('valuestddev', 48.824),
 ('rent', 48.775),
 ('raceblack', 48.414),
 ('povertypopbelowpovertylevel', 47),
 ('SourceConstituentCall', 46.805),
 ('PropertyTypeAddress', 46.805),
 ('raceother', 46.778),
 ('queuewkopen', 46.709),
 ('racehispanic', 46.083),
 ('queuewk', 44.748),
 ('SourceCitizensConnectApp', 44.231),
 ('isdescription', 44.221000000000004),
 ('povertypopwfoodstamps', 40.479)]